## Introduction

MNIST ("Modified National Institute of Standards and Technology") is the de facto “Hello World” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike.

In this competition, we aim to correctly identify digits from a dataset of tens of thousands of handwritten images. Kaggle has curated a set of tutorial-style kernels which cover everything from regression to neural networks. They hope to encourage us to experiment with different algorithms to learn first-hand what works well and how techniques compare.

## Approach

For this competition, we will be using Keras (with TensorFlow as our backend) as the main package to create a simple neural network to predict, as accurately as we can, digits from handwritten images. In particular, we will be calling the Functional Model API of Keras, and creating a 4-layered and 5-layered neural network.

Also, we will be experimenting with various optimizers: the plain vanilla Stochastic Gradient Descent optimizer and the Adam optimizer. However, there are many other parameters, such as training epochs which will we will not be experimenting with.

In addition, the choice of hidden layer units are completely arbitrary and may not be optimal. This is yet another parameter which we will not attempt to tinker with. Lastly, we introduce dropout, a form of regularisation, in our neural networks to prevent overfitting.

## Result

Following our simulations on the cross validation dataset, it appears that a 4-layered neural network, using 'Adam' as the optimizer along with a learning rate of 0.01, performs best. We proceed to introduce dropout in the model, and use the model to predict for the test set.

The test predictions (submitted to Kaggle) generated by our model predicts with an accuracy score of 97.600%, which places us at the top 55 percentile of the competition.

Importing key libraries, and reading data

In [1]:
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

In [3]:
df_train.head() # 784 features, 1 label

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Splitting into training and validation dataset

In [4]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_test = df_test.iloc[:, 0:784]

print(X_test.shape)

(28000, 784)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label, 
                                                test_size = 0.2,
                                                random_state = 1212)

X_train = X_train.as_matrix().reshape(33600, 784) #(33600, 784)
X_cv = X_cv.as_matrix().reshape(8400, 784) #(8400, 784)

X_test = X_test.as_matrix().reshape(28000, 784)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


## Data cleaning, normalization and selection

In [6]:
print((min(X_train[1]), max(X_train[1])))

(0, 255)


As the pixel intensities are currently between the range of 0 and 255, we proceed to normalize the features, using broadcasting. In addition, we proceed to convert our labels from a class vector to binary One Hot Encoded

In [7]:
# Feature Normalization 
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

# Convert labels to One Hot Encoded
num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)

In [8]:
# Printing 2 examples of labels after conversion
print(y_train[0]) # 2
print(y_train[3]) # 7

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


## Model Fitting

We proceed by fitting several simple neural network models using Keras (with TensorFlow as our backend) and collect their accuracy. The model that performs the best on the validation set will be used as the model of choice for the competition.

Model 1: Simple Neural Network with 4 layers (300, 100, 100, 200)

In our first model, we will use the Keras library to train a neural network with the activation function set as ReLu. To determine which class to output, we will rely on the SoftMax function

In [9]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [10]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [11]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model = Model(Inp, output)
model.summary() # We have 297,910 parameters to estimate

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010      
Total params: 297,910
Trainable params: 297,910
Non-trainable params: 0
_____________________________________________________

In [12]:
# Insert Hyperparameters
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(lr=learning_rate)

In [13]:
# We rely on the plain vanilla Stochastic Gradient Descent as our optimizing methodology
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [14]:
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 2s - loss: 1.8541 - accuracy: 0.4985 - val_loss: 1.0047 - val_accuracy: 0.7601
Epoch 2/20
 - 2s - loss: 0.6482 - accuracy: 0.8293 - val_loss: 0.4640 - val_accuracy: 0.8720
Epoch 3/20
 - 2s - loss: 0.4099 - accuracy: 0.8834 - val_loss: 0.3621 - val_accuracy: 0.8976
Epoch 4/20
 - 2s - loss: 0.3377 - accuracy: 0.9028 - val_loss: 0.3123 - val_accuracy: 0.9102
Epoch 5/20
 - 2s - loss: 0.2980 - accuracy: 0.9140 - val_loss: 0.2893 - val_accuracy: 0.9175
Epoch 6/20
 - 2s - loss: 0.2684 - accuracy: 0.9226 - val_loss: 0.2651 - val_accuracy: 0.9238
Epoch 7/20
 - 2s - loss: 0.2454 - accuracy: 0.9296 - val_loss: 0.2558 - val_accuracy: 0.9257
Epoch 8/20
 - 2s - loss: 0.2273 - accuracy: 0.9352 - val_loss: 0.2322 - val_accuracy: 0.9338
Epoch 9/20
 - 2s - loss: 0.2102 - accuracy: 0.9378 - val_loss: 0.2176 - val_accuracy: 0.9363
Epoch 10/20
 - 2s - loss: 0.1952 - accuracy: 0.9439 - val_loss: 0.2053 - val_accuracy: 0.9398
Epoch 11/20
 - 2s - 

Using a 4 layer neural network with:

1. 20 training epochs
2. A training batch size of 100
3. Hidden layers set as (300, 100, 100, 200)
4. Learning rate of 0.1

Achieved a training score of around 96-98% and a test score of around 95 - 97%.

Can we do better if we were to change the optimizer? To find out, we use the Adam optimizer for our second model, while maintaining the same parameter values for all other parameters.

In [15]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology
adam = keras.optimizers.Adam(lr=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 3s - loss: 0.3405 - accuracy: 0.8977 - val_loss: 0.1635 - val_accuracy: 0.9492
Epoch 2/20
 - 2s - loss: 0.1268 - accuracy: 0.9610 - val_loss: 0.1290 - val_accuracy: 0.9611
Epoch 3/20
 - 2s - loss: 0.0843 - accuracy: 0.9732 - val_loss: 0.1042 - val_accuracy: 0.9677
Epoch 4/20
 - 3s - loss: 0.0623 - accuracy: 0.9801 - val_loss: 0.1313 - val_accuracy: 0.9608
Epoch 5/20
 - 2s - loss: 0.0452 - accuracy: 0.9849 - val_loss: 0.1088 - val_accuracy: 0.9690
Epoch 6/20
 - 2s - loss: 0.0403 - accuracy: 0.9867 - val_loss: 0.0968 - val_accuracy: 0.9735
Epoch 7/20
 - 2s - loss: 0.0353 - accuracy: 0.9879 - val_loss: 0.0961 - val_accuracy: 0.9740
Epoch 8/20
 - 2s - loss: 0.0248 - accuracy: 0.9925 - val_loss: 0.1102 - val_accuracy: 0.9729
Epoch 9/20
 - 2s - loss: 0.0219 - accuracy: 0.9925 - val_loss: 0.0928 - val_accuracy: 0.9787
Epoch 10/20
 - 2s - loss: 0.0234 - accuracy: 0.9924 - val_loss: 0.1174 - val_accuracy: 0.9705
Epoch 11/20
 - 2s - 

As it turns out, it does appear to be the case that the optimizer plays a crucial part in the validation score. In particular, the model which relies on 'Adam' as its optimizer tend to perform 1.5 - 2.5% better on average. Going forward, we will use 'Adam' as our optimizer of choice.

What if we changed the learning rate from 0.1 to 0.01, or 0.5? Will it have any impact on the accuracy?
Model 2A

In [17]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.01
adam = keras.optimizers.Adam(lr=learning_rate)
model2a = Model(Inp, output)

model2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
history2a = model2a.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                        verbose = 2,
                        validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
 - 3s - loss: 0.3382 - accuracy: 0.8982 - val_loss: 0.1951 - val_accuracy: 0.9415
Epoch 2/20
 - 2s - loss: 0.1244 - accuracy: 0.9616 - val_loss: 0.1244 - val_accuracy: 0.9626
Epoch 3/20
 - 2s - loss: 0.0834 - accuracy: 0.9742 - val_loss: 0.0918 - val_accuracy: 0.9724
Epoch 4/20
 - 2s - loss: 0.0594 - accuracy: 0.9808 - val_loss: 0.1027 - val_accuracy: 0.9701
Epoch 5/20
 - 2s - loss: 0.0468 - accuracy: 0.9849 - val_loss: 0.0961 - val_accuracy: 0.9733
Epoch 6/20
 - 2s - loss: 0.0328 - accuracy: 0.9898 - val_loss: 0.1082 - val_accuracy: 0.9707
Epoch 7/20
 - 2s - loss: 0.0327 - accuracy: 0.9890 - val_loss: 0.1212 - val_accuracy: 0.9701
Epoch 8/20
 - 2s - loss: 0.0263 - accuracy: 0.9915 - val_loss: 0.1058 - val_accuracy: 0.9742
Epoch 9/20
 - 2s - loss: 0.0215 - accuracy: 0.9935 - val_loss: 0.1092 - val_accuracy: 0.9710
Epoch 10/20
 - 2s - loss: 0.0245 - accuracy: 0.9924 - val_loss: 0.1133 - val_accuracy: 0.9715
Epoch 11/20
 - 2s - 

Model 2B

In [19]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.5
adam = keras.optimizers.Adam(lr=learning_rate)
model2b = Model(Inp, output)

model2b.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
history2b = model2b.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                            validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 3s 83us/step - loss: 0.3336 - accuracy: 0.9034 - val_loss: 0.1445 - val_accuracy: 0.9580
Epoch 2/20
33600/33600 [==============================] - 2s 74us/step - loss: 0.1220 - accuracy: 0.9628 - val_loss: 0.1310 - val_accuracy: 0.9598
Epoch 3/20
33600/33600 [==============================] - 2s 74us/step - loss: 0.0783 - accuracy: 0.9753 - val_loss: 0.0977 - val_accuracy: 0.9706
Epoch 4/20
33600/33600 [==============================] - 2s 74us/step - loss: 0.0560 - accuracy: 0.9828 - val_loss: 0.1107 - val_accuracy: 0.9686
Epoch 5/20
33600/33600 [==============================] - 2s 73us/step - loss: 0.0495 - accuracy: 0.9833 - val_loss: 0.0896 - val_accuracy: 0.9745
Epoch 6/20
33600/33600 [==============================] - 2s 74us/step - loss: 0.0380 - accuracy: 0.9881 - val_loss: 0.0952 - val_accuracy: 0.9720
Epoch 7/20
33600/33600 [==============================] - 2s 74us/ste

The accuracy, as measured by the 3 different learning rates 0.01, 0.1 and 0.5 are around 98%, 97% and 98% respectively. As there are no considerable gains by changing the learning rates, we stick with the default learning rate of 0.01.

We proceed to fit a neural network with 5 hidden layers with the features in the hidden layer set as (300, 100, 100, 100, 200) respectively. To ensure that the two models are comparable, we will set the training epochs as 20, and the training batch size as 100.

In [21]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 100
n_hidden_5 = 200
num_digits = 10

In [22]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
x = Dense(n_hidden_5, activation='relu', name = "Hidden_Layer_5")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [23]:
# Our model would have '7' layers - input layer, 5 hidden layer and 1 output layer
model3 = Model(Inp, output)
model3.summary() # We have 308,010 parameters to estimate

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_4 (Dense)       (None, 100)               10100     
_________________________________________________________________
Hidden_Layer_5 (Dense)       (None, 200)               20200     
_________________________________________________________________
Output_Layer (Dense)         (None, 10)                2010

In [24]:
# We rely on 'Adam' as our optimizing methodology
adam = keras.optimizers.Adam(lr=0.01)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
history3 = model3.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 3s 84us/step - loss: 0.3602 - accuracy: 0.8930 - val_loss: 0.1909 - val_accuracy: 0.9440
Epoch 2/20
33600/33600 [==============================] - 3s 76us/step - loss: 0.1276 - accuracy: 0.9603 - val_loss: 0.1247 - val_accuracy: 0.9630
Epoch 3/20
33600/33600 [==============================] - 3s 75us/step - loss: 0.0848 - accuracy: 0.9742 - val_loss: 0.1114 - val_accuracy: 0.9662
Epoch 4/20
33600/33600 [==============================] - 3s 76us/step - loss: 0.0619 - accuracy: 0.9805 - val_loss: 0.1020 - val_accuracy: 0.9704
Epoch 5/20
33600/33600 [==============================] - 3s 76us/step - loss: 0.0499 - accuracy: 0.9838 - val_loss: 0.0970 - val_accuracy: 0.9718
Epoch 6/20
33600/33600 [==============================] - 3s 75us/step - loss: 0.0411 - accuracy: 0.9874 - val_loss: 0.0905 - val_accuracy: 0.9750
Epoch 7/20
33600/33600 [==============================] - 3s 75us/ste

Compared to our first model, adding an additional layer did not significantly improve the accuracy from our previous model. However, there are computational costs (in terms of complexity) in implementing an additional layer in our neural network. Given that the benefits of an additional layer are low while the costs are high, we will stick with the 4 layer neural network.

We now proceed to include dropout (dropout rate of 0.3) in our second model to prevent overfitting.

In [26]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [27]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [28]:
# Our model would have '6' layers - input layer, 4 hidden layer and 1 output layer
model4 = Model(Inp, output)
model4.summary() # We have 297,910 parameters to estimate

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 784)               0         
_________________________________________________________________
Hidden_Layer_1 (Dense)       (None, 300)               235500    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
Hidden_Layer_2 (Dense)       (None, 100)               30100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
Hidden_Layer_3 (Dense)       (None, 100)               10100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0   

In [29]:
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
history = model4.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_cv, y_cv))

Train on 33600 samples, validate on 8400 samples
Epoch 1/20
33600/33600 [==============================] - 3s 95us/step - loss: 0.5821 - accuracy: 0.8137 - val_loss: 0.1895 - val_accuracy: 0.9423
Epoch 2/20
33600/33600 [==============================] - 3s 84us/step - loss: 0.2281 - accuracy: 0.9334 - val_loss: 0.1350 - val_accuracy: 0.9606
Epoch 3/20
33600/33600 [==============================] - 3s 85us/step - loss: 0.1768 - accuracy: 0.9492 - val_loss: 0.1135 - val_accuracy: 0.9664
Epoch 4/20
33600/33600 [==============================] - 3s 84us/step - loss: 0.1422 - accuracy: 0.9585 - val_loss: 0.1071 - val_accuracy: 0.9693
Epoch 5/20
33600/33600 [==============================] - 3s 84us/step - loss: 0.1218 - accuracy: 0.9634 - val_loss: 0.0935 - val_accuracy: 0.9742
Epoch 6/20
33600/33600 [==============================] - 3s 84us/step - loss: 0.1063 - accuracy: 0.9680 - val_loss: 0.0861 - val_accuracy: 0.9744
Epoch 7/20
33600/33600 [==============================] - 3s 84us/ste

With a validation score of close to 98%, we proceed to use this model to predict for the test set.

In [31]:
test_pred = pd.DataFrame(model4.predict(X_test, batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [32]:
test_pred.to_csv('mnist_submission.csv', index = False)

Using this model, we are able to achieve a score of 0.976, which places us at the top 55th percentile!